# PAL case study target analysis
This case study concerns the fluorescence measured from the PAL mutant of a cyanobacterium, which contains only photosystem I (PSI) and photosystem II (PSII). With PSI the difference between open and closed reaction centers (RC) is negligible in the fluorescence, because both the oxidized and reduced RC efficiently quench the excitations. However, with PSII there is a huge difference, with closed RCs exhibiting a huge amount of recombination fluorescence. The measurement and the global analysis are described in ([Tian et al. 2013][tian2013]).
The target analysis is based upon the model explained in ([van Stokkum 2018][vanstokkum2018]).

[tian2013]: https://doi.org/10.1039/C3CP43813A "Tian L, Farooq S, van Amerongen H (2013) Probing the picosecond kinetics of the photosystem II core complex in vivo. Physical Chemistry Chemical Physics 15 (9):3146-3154."
[vanstokkum2018]: https://www.few.vu.nl/~ivo/pub/2018/2018GTAofLHandPQinvivo.pdf "Systems biophysics: Global and target analysis of light harvesting and photochemical quenching in vivo. In: Croce R, van Grondelle R, van Amerongen H, van Stokkum I (eds) Light Harvesting in Photosynthesis. CRC Press, Boca Raton, pp 467-482."

# Simultaneous Target Analysis of PAL in the open and in the closed state

### Read PAL data

In [ ]:
from pyglotaran_extras import plot_data_overview

dataset_open_path = "PAL_700_ma_tr2_gain50_10uW_590nmfilter_21C_400nm_AV_bc_sh_sel_620_830.ascii"
dataset_closed_path = (
    "PAL_DCMU_80uM_WL_SF_700_ma_tr2_gain50_100uW_590nmfilter_21C_400nm_AV_bc_sh_sel_620_830.ascii"
)
plot_data_overview(dataset_open_path, nr_of_data_svd_vectors=3, title="PAL open")
plot_data_overview(dataset_closed_path, nr_of_data_svd_vectors=3, title="PAL closed");

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

PAL_open_closed_target_model_path = "models/PAL-open-closed-target_exercise-model.yml"
PAL_open_closed_target_parameters_path = "models/PAL-open-closed-target_exercise-parameters.yml"

#### Model file

In [ ]:
display_file(PAL_open_closed_target_model_path, syntax="yaml")

#### Parameters file

Exercise 1: Make sure you thoroughly understand the target model that is applied here. Choose the right residual_function.

Describe the model qualitatively in your own words.

Exercise 2: Study the parameters file carefully. Insert the missing starting values from GTAofLHandPQinvivo.pdf. Comment on the amount of free parameters. Are they all indispensable?

In [ ]:
display_file(PAL_open_closed_target_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
from glotaran.project.scheme import Scheme

PAL_open_closed_target_scheme = Scheme(
    PAL_open_closed_target_model_path,
    PAL_open_closed_target_parameters_path,
    {"dataset_open": dataset_open_path, "dataset_closed": dataset_closed_path},
    maximum_number_function_evaluations=5,
)

In [ ]:
from glotaran.optimization.optimize import optimize

PAL_open_closed_target_result = optimize(PAL_open_closed_target_scheme)

In [ ]:
PAL_open_closed_target_result

In [ ]:
PAL_open_closed_target_result.data["dataset_open"]

In [ ]:
compartments = PAL_open_closed_target_scheme.model.initial_concentration["input1"].compartments

PAL_open_closed_target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
PAL_open_closed_target_scheme.model.k_matrix["km_open"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
PAL_open_closed_target_scheme.model.k_matrix["km_closed"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
from show_a_matrix import show_a_matrix

show_a_matrix(PAL_open_closed_target_result)

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(
    PAL_open_closed_target_result.data["dataset_open"], wavelength_range=(670, 750)
)
fig3tr, axes = plot_fitted_traces(
    PAL_open_closed_target_result, wavelengths, linlog=True, linthresh=50
)

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    PAL_open_closed_target_result.data["dataset_closed"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=50,
);

In [ ]:
plot_overview(PAL_open_closed_target_result.data["dataset_open"], linlog=False, figure_only=False);

Exercise 3: Comment on the interpretation of the SAS, with the help of the literature, and relate this to the target model used.

## Dispersion curves

In [ ]:
PAL_open_closed_target_result.data["dataset_open"].data.sel(time=slice(90, 150)).plot()
PAL_open_closed_target_result.data["dataset_open"].irf_center_location.plot.line(
    x="spectral", add_legend=False
)

In [ ]:
PAL_open_closed_target_result.data["dataset_closed"].data.sel(time=slice(110, 170)).plot()
PAL_open_closed_target_result.data["dataset_closed"].irf_center_location.plot.line(
    x="spectral", add_legend=False
)

In [ ]:
PAL_open_closed_target_result.optimized_parameters